## Workflow zum Abgleich und Import von Ortsdaten aus iDAI.gazetteer zu Wikidata

Das Notebook beschreibt den in FAIR.rdm praktizierten Workflow zum Abgleich von Ortsdaten im iDAI.gazetteer mit Wikidata.

Der Python-Code unterstützt beim Anlegen neuer Orte mittels Quick Statements.

In [ ]:
import pandas as pd
import requests
import time

Als erstes muss mit OpenRefine ein Agleich durchgeführt werden, welche Daten sich bereits in Wikidata befinden. Die Wikidata URIs sind dann in einer separaten Spalten anzugeben (Abgleichen/Add columns with URLs of matched entities).

Diese Tablle kann dann für den folgenden Code exportiert werden.

In [ ]:
df_toimport = pd.read_excel("Gazetteer_To_Wikidata_Mapping_2025-07-11.xlsx", sheet_name="wikidata")
df_toimport = df_toimport.fillna("absent")

df_toimport = df_toimport[df_toimport['Wikidata_URL'] == "absent"]
df_toimport = df_toimport.reset_index(drop=True)
df_toimport.drop('Wikidata_URL', axis=1, inplace=True)

#Teste das Skript mit einem Auszug der Daten:
#df_test = df_toimport[14:16]

In [ ]:
#Sollten in der ersten Spalte Gazetteer-URIs stehen, wird hier die Gazetteer-ID extrahiert.
def get_gaz_id(uri):
    parts = uri.split("/")
    gazid = parts[-1]
    return gazid

In [ ]:
#Diese Funktion fragt den Namens eines Ortes im Gazetteer ab.
def get_gaz_title(gaz_id):
    gaz_id = gaz_id.split("/")[-1]
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'prefName' in data:
            title = data['prefName']['title']
            return title
        else:
            title = "ERROR_no_title!!!"
            return title

In [ ]:
#Diese Funkton übernimmmt den Namen eines Orts und setzt ihn als Wikidata-Label ein.
def get_label(row):
    label = row['PlaceName']
    return label

In [ ]:
#Diese Funktion übernimmt den Type im Gazetteer als Beschreibung in Wikidata. Sie muss ggf. erweitert werden.

def get_gaz_type(gaz_id):
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        types = data['types']
        title = data['prefName']['title']
        if "Keramik" in title:
            type = "archäologischer Keramikstil"
        elif 'archaeological-area' in types:
            type = "Archäologischer Bereich"
        elif 'archaeological-site' in types:
            type = "Archäologischer Ort"
        else:
            type = types[0]
        return type
    else:
        type = "ERROR_no_type!!!"
        return type

In [ ]:
#Diese Funktion bestimmt die Aussage "ist ein(e) ..." (P31) für Wikidata.

def get_p31(description):
    if description == "Archäologischer Bereich":
        p31 = "Q839954 (Archäologische Stätte)"
        return p31
    elif description == "archäologischer Keramikstil":
        p31 = "Q24017852 (Keramikstil)"
        return p31
    elif description == "Archäologischer Ort":
        p31 = "Q839954 (Archäologische Stätte)"
        return p31
    else:
        p31 = "set manually"
        return p31

In [ ]:
#Im Fall von Polygonen wird hiermit der Mittelpunkt des Polygons berechnet und dieser in Wikidata immportiert.

def polygon_centroid(coords):
    # Unwrap the nested list structure
    points = coords[0][0]
    x_list = [p[0] for p in points]
    y_list = [p[1] for p in points]
    n = len(points) - 1  # last point is same as first

    area = 0.0
    cx = 0.0
    cy = 0.0

    for i in range(n):
        factor = (x_list[i] * y_list[i+1] - x_list[i+1] * y_list[i])
        area += factor
        cx += (x_list[i] + x_list[i+1]) * factor
        cy += (y_list[i] + y_list[i+1]) * factor

    area *= 0.5
    if area == 0:
        # fallback: average of points
        return (sum(x_list)/len(x_list), sum(y_list)/len(y_list))
    cx /= (6.0 * area)
    cy /= (6.0 * area)
    return (cx, cy)

In [ ]:
#Diese Funktion fragt die Latitude eines Ortes im Gazetteer ab.

def get_latitude(gaz_id):
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print(data)
        location = data['prefLocation']
        if 'coordinates' in location:
            lat = data['prefLocation']["coordinates"][0]
        elif 'shape' in location:
            coords = location['shape']
            center = polygon_centroid(coords)
            lat = center[0]
        else:
            lat = "ERROR_no_coordinates!!!"
        return lat

In [ ]:
#Diese Funktion fragt die Longitude eines Ortes im Gazetteer ab.

def get_longitude(gaz_id):
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'prefLocation' in data and 'coordinates' in data['prefLocation']:
            long = data['prefLocation']["coordinates"][-1]
        elif 'shape' in data['prefLocation']:
            coords = data['prefLocation']['shape']
            center = polygon_centroid(coords)
            long = center[1]
        else:
            long = "ERROR_no_coordinates!!!"
        return long

In [ ]:
#Diese Funktion  bestimmt das Land eines Ortes anhand der Latitude und Longitude. Sie nutzt den Nominatim-Dienst von OpenStreetMap.

def get_country_from_latlon(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lon,
        "lon": lat,
        "format": "json",
        "zoom": 3,  # 3 = country level
        "addressdetails": 1
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; CopilotBot/1.0)"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        address = data.get("address", {})
        country = address.get("country")
        return country
    else:
        return None

In [ ]:
#Main Loop

labels = []
descriptions = []
type = []
lat = []
long = []
states = []

for index, row in df_toimport.iterrows():
    label = get_label(row)
    labels.append(label)
    gaz_id = get_gaz_id(row['GazetteerID'])
    description = get_gaz_type(gaz_id)
    descriptions.append(description)
    p31_value = get_p31(description)
    type.append(p31_value)
    latitude = get_latitude(gaz_id)
    longitude = get_longitude(gaz_id)
    #latidute und longitude dürfen nur fünt Nachkommastellen haben
    latitude = round(latitude, 5)
    longitude = round(longitude, 5)
    lat.append(latitude)
    long.append(longitude)
    country = get_country_from_latlon(latitude, longitude)
    states.append(country)
    #Wait for three seconds to avoid rate limiting
    time.sleep(3)

print(labels)
print(descriptions)
print(type)
print(lat)
print(long)
print(states)

In [ ]:
#export to csv
df_toexport = pd.DataFrame({
    'GazetteerID': df_toimport['GazetteerID'],
    'Label': labels,
    'Description': descriptions,
    'Type': type,
    'Latitude': lat,
    'Longitude': long,
    'Country': states
})

df_toexport.to_csv("Wikidata_information.csv", index=False, encoding='utf-8', sep=';')

Jetzt können an der CSV nötige Änderungen vorgenommen werden und Einträge manuell ergänzt werden.

Dananch können mit folgendem Code die QuickStatements erzeugt werden:

In [ ]:
#Das Mapping der Länder zu Wikidata Q-IDs muss ggf. erweitert werden.

country_mapping = {
    "ليبيا": "Q1016",   # Libyen
    "Sudan": "Q1049",
    "مصر": "Q79",       # Ägypten
    "Niger": "Q1032",
    "Mali": "Q912",
    "Algérie ⵍⵣⵣⴰⵢⴻⵔ الجزائر": "Q262",
    "République démocratique du Congo": "Q974",
    "Congo": "Q971",
    "Cameroun": "Q1009",
    "السودان":"Q1049" #Sudan
}

df = pd.read_csv('Wikidata_information.csv', sep=";")

qs_lines = []

for _, row in df.iterrows():
    label = str(row["Label"]).strip()
    desc_de = str(row["Description"]).strip()
    desc_en = "archaeological site"
    type_qid = row["Type"].split()[0].strip()
    lat = row["Latitude"]
    lon = row["Longitude"]
    country = country_mapping.get(str(row["Country"]).strip(), None)
    #country = row["Country"]
    gazetteer_url = row["GazetteerID"]

    qs_lines.append("CREATE")
    qs_lines.append(f'LAST|Lde|"{label}"')
    qs_lines.append(f'LAST|Den|"{desc_en}"')
    qs_lines.append(f'LAST|Dde|"{desc_de}"')
    qs_lines.append(f'LAST|P31|{type_qid}')
    qs_lines.append(f'LAST|P625|@{lon}/{lat}|S854|"{gazetteer_url}"')
    if country:
        qs_lines.append(f'LAST|P17|{country}')

    qs_lines.append("")  # Leerzeile für neuen Eintrag

# Datei speichern
with open('quickstatements_ortsimport.txt', "w", encoding="utf-8") as f:
    f.write("\n".join(qs_lines))

Gehe zu https://quickstatements.toolforge.org/

Melde dich mit deinem Wikidata-Account an.

Klicke auf „New batch“.

Wenn hier eine Meldung angezeigt wird, dass das aktuelle Wiki-Konto nicht den Status "autoconfirmed" hat, müssen noch manuell 50 Änderungen in Wikidata durchgeführt werden. Dann wird das Konto automatisch freigeschaltet.

Füge den Inhalt der Datei ein oder lade sie hoch.

Klicke auf „Run“ oder „Only add“ (zum Testen).

Entwickelt von Lukas Lammers im Projekt FAIR.rdm, teil des DFG-geförderten SPP 2143 "Entangled Africa".

Mit Unterstützung von GitHub Copilot (KI-Assistent) bei der Code-Entwicklung und -Optimierung.

Version 1.0, 15.07.2025